In [45]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from Utils import create_features
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization

# Load the data
file_path = 'date-wise-retail-selling-price-of-petrol-and-diesel-in-metro-cities.xlsx'
data = pd.read_excel(file_path,parse_dates=True,index_col='date')
# data.info()
data = create_features(data)


cols_to_drop = ['unit','note']
data.drop(columns=cols_to_drop,inplace=True)
data.head()


cat_cols = data.select_dtypes(include="object").columns
print("Categorical Columns : ",cat_cols)


from sklearn.preprocessing  import OneHotEncoder
encoder = OneHotEncoder()



# # Sort the data by date
data = data.sort_values(by='date')

target_column = ['value']

# Prepare the dataset
X = data.drop(columns=target_column)
y = data[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
print(X_train.columns,X_test.columns)




Categorical Columns :  Index(['city', 'fuel_type'], dtype='object')
Index(['city', 'fuel_type', 'year', 'month', 'day'], dtype='object') Index(['city', 'fuel_type', 'year', 'month', 'day'], dtype='object')


In [46]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

cat_pipe = Pipeline(steps=[
    ("Encoder",OneHotEncoder())
])

transformer = ColumnTransformer([
    ("Categorical_Encoding",cat_pipe,cat_cols)
],remainder="passthrough")

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

In [47]:
for col in X_test.select_dtypes(include="object").columns:
    print(f"{col} : {data[col].unique()}")

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

In [48]:
X_test.shape,X_train.shape

((4106, 9), (16422, 9))

In [51]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=500)
model.fit(X_train,y_train)


c:\Users\chandana\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=500)

In [52]:

# Predict the test set
y_pred = model.predict(X_test)

# Calculate accuracy metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Mean Squared Error: {mse:.2f}')
print(f'R² Score: {r2:.2f}')


Mean Absolute Error: 2.35
Mean Squared Error: 21.84
R² Score: 0.39


In [53]:
import joblib

# Save the trained model
joblib.dump(model, 'model.pkl')
joblib.dump(transformer,'preprocessor.pkl')

['preprocessor.pkl']

In [57]:
def predict_fuel_price(day, month, year, city, fuel_type):
    data = {'city':[city],
            'fuel_type':[fuel_type],
            'month':[month],
            'year':[year],
            'day':[day]} 

    input_df = pd.DataFrame(data)
    input_df.head()
    transformed_data =transformer.transform(input_df)
    predicted_price = model.predict(transformed_data)
    return predicted_price[0]

# Example usage
predicted_price = predict_fuel_price(day=2, month=8, year=2025, city='Mumbai', fuel_type='Diesel')
print(f'Predicted price: {predicted_price:.2f}')

Predicted price: 94.27
